In [ ]:
from dotenv import load_dotenv
from openai import OpenAI
import os

load_dotenv(override=True)
google_api_key = os.getenv('GOOGLE_API_KEY')
openai = OpenAI()

In [11]:
tools = [
    {
        "type": "function",
        "name": "get_population",
        "description": "Get the population of a city",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "City name"
                }
            },
            "required": ["city"]
        }
    },
]


In [12]:
def get_population(city: str) -> int:
    """
    Mock population lookup
    """
    population_db = {
        "mumbai": 12442373,
        "pune": 3124458,
        "delhi": 16787941,
        "bangalore": 8443675
    }

    print("inside get_population function", city, city.lower() not in population_db)
    if city.lower() not in population_db:
        raise ValueError("City not found")

    return population_db.get(city.lower(), 0)


In [27]:
import time
from typing import Any, Dict, List

TOOL_TRACE_LOG: List[Dict[str, Any]] = []

def trace_tool_call(tool_name, fn, **kwargs):
    start = time.perf_counter()
    status = "success"

    try:
        output = fn(**kwargs)
    except Exception as e:
        output = str(e)
        status = "error"

    latency_ms = round((time.perf_counter() - start) * 1000, 2)

    TOOL_TRACE_LOG.append({
        "tool": tool_name,
        "inputs": kwargs,
        "output": output,
        "latency_ms": latency_ms,
        "status": status
    })

    return output


In [25]:
def execute_tool_with_tracing(tool_name, args):
    if tool_name == "get_population":
        return trace_tool_call(
            "get_population",
            get_population,
            city=args["city"]
        )

    raise ValueError("Unknown tool")


In [33]:
from openai import OpenAI
import json
client = OpenAI()

def agent_with_tracing(user_input: str):
    # 1️⃣ Initial model call
    response = client.responses.create(
        model="gpt-4.1-mini",
        input=user_input,
        tools=tools,
        tool_choice="auto"
    )

    tool_result_inputs = []

    for item in response.output:
        if item.type == "function_call":
            tool_name = item.name
            args = item.arguments

            print(f"[TRACE] Tool requested → {tool_name} {args}")

            
            result = execute_tool_with_tracing(tool_name, json.loads(args))

            print(f"[TRACE] Tool result → {result}")

            # 🔑 Tool results are INPUT ITEMS
            tool_result_inputs.append({
                "type": "function_call_output",
                "call_id": item.call_id,
                "output": str(result)
            })

    # 2️⃣ If tools were called, continue the response
    if tool_result_inputs:
        final = client.responses.create(
            model="gpt-4.1-mini",
            previous_response_id=response.id,
            input=tool_result_inputs
        )
        return final.output_text

    # 3️⃣ No tools → return directly
    return response.output_text



result = agent_with_tracing("Mumbai's population")
print("result", result)

[TRACE] Tool requested → get_population {"city":"Mumbai"}
inside get_population function Mumbai False
[TRACE] Tool result → 12442373
result The population of Mumbai is approximately 12,442,373.


In [ ]:
import gradio as gr
import pandas as pd

def get_trace_table():
    if not TOOL_TRACE_LOG:
        return pd.DataFrame(columns=["tool", "inputs", "output", "latency_ms", "status"])
    return pd.DataFrame(TOOL_TRACE_LOG)

with gr.Blocks(title="Tool Tracing Dashboard") as demo:
    gr.Markdown("## 🔍 Tool Invocation Trace Dashboard")

    with gr.Row():
        user_input = gr.Textbox(
            label="User Query",
            placeholder="Ask something that triggers tools..."
        )
        run_btn = gr.Button("Run Agent")

    output = gr.Textbox(label="Agent Output")

    trace_table = gr.Dataframe(
        headers=["tool", "inputs", "output", "latency_ms", "status"],
        interactive=False
    )

    def run_and_refresh(query):
        result = agent_with_tracing(query)
        print("result", result)
        return result, get_trace_table()

    run_btn.click(
        fn=run_and_refresh,
        inputs=user_input,
        outputs=[output, trace_table]
    )

demo.launch()


* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.


[TRACE] Tool requested → get_population {"city":"Mumbai"}
inside get_population function Mumbai False
[TRACE] Tool result → 12442373
result The population of Mumbai is approximately 12,442,373.
